In [1]:
import pandas as pd
import csv
import json
import requests
from datetime import datetime
import itertools
from collections import Counter

In [83]:
url = 'http://data.parlameter.si/v1/getVotesOfSessionTable/6684/'
data = pd.read_json(url)
coalition = requests.get('https://data.parlameter.si/v1/getCoalitionPGs').json()['coalition']
orgs = requests.get('https://data.parlameter.si/v1/getAllPGsExt/')
data['option_ni'] = 0
data['option_za'] = 0
data['option_proti'] = 0
data['option_kvorum'] = 0
data.loc[data['option'] == 'ni', 'option_ni'] = 1
data.loc[data['option'] == 'za', 'option_za'] = 1
data.loc[data['option'] == 'proti', 'option_proti'] = 1
data.loc[data['option'] == 'kvorum', 'option_kvorum'] = 1
data['voter_unit'] = 1
data['is_coalition'] = 0
data.loc[data['voterparty'].isin(coalition), 'is_coalition'] = 1

def getPercent(a, b, c, d=None):
    a = 0 if pd.isnull(a) else a
    b = 0 if pd.isnull(b) else b
    c = 0 if pd.isnull(c) else c
    if d:
        d = 0 if pd.isnull(d) else d
        devider = float(sum([a, b, c, d]))
        if devider:
            return max(a, b, c, d) / devider *100
        else:
            return 0
    else:
        devider = float(sum([a, b, c]))
        if devider:
            return max(a, b, c) / devider *100
        else:
            return 0

In [84]:
#za proti ni kvorum
all_votes = data.groupby('vote_id').sum()
#all_votes[['option_ni', 'option_za', 'option_proti', 'option_kvorum']]

In [85]:
all_votes['max_option_percent'] = all_votes.apply(lambda row: getOptions(row, 'ps'), axis=1)

In [86]:
all_votes[['option_za', 'option_kvorum', 'option_proti','option_ni', 'max_option_percent']]


,option_za,option_kvorum,option_proti,option_ni,max_option_percent
vote_id,,,,,
6509,66,20,0,4,"{""max"": {""maxOptPerc"": 73.333333333333329, ""ma..."
6510,88,0,0,2,"{""max"": {""maxOptPerc"": 97.777777777777771, ""ma..."
6511,64,0,21,5,"{""max"": {""maxOptPerc"": 71.111111111111114, ""ma..."
6512,26,0,59,5,"{""max"": {""maxOptPerc"": 65.555555555555557, ""ma..."
6513,84,0,0,6,"{""max"": {""maxOptPerc"": 93.333333333333329, ""ma..."


In [87]:
#members kvorum
def getMPsList(row, proti):
    try:
        print "normal"
        return json.dumps(list(proti[row.name].reset_index()['voter']))
        
    except:
        try:
            print "one line"
            print json.dumps(list(proti.values[0]))
            return json.dumps(list(proti.values[0]))
            
        except:
            print "ni se ql"
            json.dumps([])
    
def getPGsList(row, proti):
    try:
        return str(dict(Counter(list(proti[row.name].reset_index()['voterparty']))))
    except:
        return []

    #return row.name
m_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voter"])

pg_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])

all_votes['m_proti'] = all_votes.apply(lambda row: getMPsList(row, m_proti),axis=1)
all_votes['m_za'] = all_votes.apply(lambda row: getMPsList(row, m_za),axis=1)
all_votes['m_ni'] = all_votes.apply(lambda row: getMPsList(row, m_ni),axis=1)
all_votes['m_kvorum'] = all_votes.apply(lambda row: getMPsList(row, m_kvorum),axis=1)

all_votes['pg_proti'] = all_votes.apply(lambda row: getPGsList(row, pg_proti),axis=1)
all_votes['pg_za'] = all_votes.apply(lambda row: getPGsList(row, pg_za),axis=1)
all_votes['pg_ni'] = all_votes.apply(lambda row: getPGsList(row, pg_ni),axis=1)
all_votes['pg_kvorum'] = all_votes.apply(lambda row: getPGsList(row, pg_kvorum),axis=1)

#list(proti[6669].reset_index()['voter'])

normal
one line
ni se ql
normal
one line
ni se ql
normal
normal
normal
one line
ni se ql
normal
normal
normal
normal
normal
normal
normal
normal
normal
normal
normal
one line
[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53, 54, 55, 64, 65, 66, 75, 78, 91]
normal
one line
[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53, 54, 55, 64, 65, 66, 75, 78, 91]
normal
one line
[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53, 54, 55, 64, 65, 66, 75, 78, 91]
normal
one line
[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53, 54, 55, 64, 65, 66, 75, 78, 91]
normal
one line
[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53, 54, 55, 64, 65, 66, 75, 78, 91]


In [5]:
def getOptions(row, side):
    maxOptionPercent = getPercent(row['option_za'],
                                  row['option_proti'],
                                  row['option_kvorum'],
                                  row['option_ni'])
    stats = {'for': row['option_za'],
             'against': row['option_proti'],
             'abstain': row['option_kvorum'],
             'not_present': row['option_ni']}
    max_opt = max(stats, key=stats.get)
    max_ids = [key for key, val in stats.iteritems() if val == max(stats.values())]
    if len(max_ids) > 1:
        if 'not_present' in max_ids:
            max_ids.remove('not_present')
            if len(max_ids) > 1:
                max_vote = 'cant_compute'
            else:
                max_vote = max_ids[0]
        else:
            max_vote = 'cant_compute'
    else:
        max_vote = max_ids[0]
    if side == 'oppo':
        outliers = [opt for opt in ['for', 'against'] if stats[opt]]
    else:
        outliers = [opt for opt in ['abstain', 'for', 'against'] if stats[opt]]
    for opt in max_ids:
        if opt in outliers:
            outliers.remove(opt)
    return json.dumps({'votes': {
                             'for': row['option_za'],
                             'against': row['option_proti'],
                             'abstain': row['option_kvorum'],
                             'not_present': row['option_ni'],
                             },
                       'max': {
                           'max_opt': max_vote,
                           'maxOptPerc': maxOptionPercent
                           },
                       'outliers': outliers})
all_votes['coal'] = data[data.is_coalition==1].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'coal'), axis=1)
all_votes['oppo'] = data[data.is_coalition==0].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'oppo'), axis=1)

In [75]:
#for i in data.voterparty.unique():
a = data.groupby(['vote_id', 'voterparty']).sum().apply(lambda row: getOptions(row, 'ps'), axis=1)

In [81]:
a[6668].keys()

Int64Index([1, 2, 3, 5, 6, 7, 8, 108, 109, 111], dtype='int64', name=u'voterparty')

In [6]:
def getIntraDisunion(row):
    maxOptionPercent = getPercent(row['option_za'],
                                  row['option_proti'],
                                  row['option_kvorum'])
    return 100 - maxOptionPercent

In [8]:
partyIntryDisunion = data.groupby(['vote_id', 'voterparty']).sum().apply(lambda row: getIntraDisunion(row), axis=1)

In [9]:
partyIntryDisunion

vote_id  voterparty
6668     1              0.000000
         2              0.000000
         3              0.000000
         5              0.000000
         6              0.000000
         7              0.000000
         8             33.333333
         108            0.000000
         109            0.000000
         111            0.000000
6669     1              0.000000
         2              0.000000
         3             11.111111
         5              0.000000
         6              0.000000
         7              0.000000
         8              0.000000
         108            0.000000
         109           25.000000
         111            0.000000
6670     1              0.000000
         2              0.000000
         3              0.000000
         5              5.882353
         6              0.000000
         7              0.000000
         8              0.000000
         108            0.000000
         109           25.000000
         111           

In [15]:
coalInter = data[data.is_coalition == 1].groupby(['vote_id']).sum().apply(lambda row: getIntraDisunion(row), axis=1)
oppoInter = data[data.is_coalition == 0].groupby(['vote_id']).sum().apply(lambda row: getIntraDisunion(row), axis=1)
allInter = data.groupby(['vote_id']).sum().apply(lambda row: getIntraDisunion(row), axis=1)

In [19]:
allInter

vote_id
6668    30.379747
6669    47.435897
6670    32.051282
6671    32.911392
6672    34.210526
6673    34.177215
6674    33.333333
6675    14.285714
6678    37.974684
6680    38.461538
6681    15.189873
6682    38.750000
6683    49.367089
6684    38.750000
6686     2.941176
6687     4.166667
6688    29.577465
6689    28.169014
6690    27.777778
6691    29.577465
6692     0.000000
6693    38.356164
6694    28.985507
6695     1.408451
6696    18.055556
6698    33.333333
6700    32.876712
6701    44.444444
6702    29.166667
6703    26.388889
          ...    
6712    20.895522
6713    32.835821
6715    12.121212
6716    24.615385
6717    31.818182
6718    25.757576
6719    10.769231
6720    14.285714
6722    31.168831
6723    18.644068
6724    20.000000
6725    33.898305
6726    20.338983
6727    20.000000
6728    20.000000
6729    21.666667
6730    15.000000
6731    23.728814
6732    16.666667
6733    19.298246
6734     3.750000
6735    25.641026
6736    30.769231
6737    30.769231
67

In [29]:
all_votes['m_za'] = all_votes.apply(lambda row: getMPsList(row, m_za),axis=1)

voter,0,1,2,3,4,5,6,7,8,9,...,79,80,81,83,84,85,86,87,88,89
vote_id,,,,,,,,,,,,,,,,,,,,,
7146,91,89,88,87,86,85,84,92,83,82,...,14,12,11,9,7,5,4,3,2,95


In [68]:
data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voter"]).values[0]

array([  91,   89,   88,   87,   86,   85,   84,   92,   83,   82,   81,
         78, 1355,   79,   77,   76,   75,   74,   73,   72,   71,   70,
       1357,   69,   68,   67,   65,   64, 2934,   63,   62,   61,   58,
         57,   55,   54, 2933,   53,   52,   51,   50,   49,   48,   47,
       1354,   96,   45,   44,   43,   42,   41,   40,   39,   37,   35,
         34,   33,   32,   31,   30,   29,   27,   26,   25,   24,   23,
         21,   19,   18,   17,   16,   15,   14,   12,   11,    9,    7,
          5,    4,    3,    2,   95])

In [88]:
all_votes

,id,orgvoter,result,session_id,voter,voterparty,option_ni,option_za,option_proti,option_kvorum,...,is_coalition,max_option_percent,m_proti,m_za,m_ni,m_kvorum,pg_proti,pg_za,pg_ni,pg_kvorum
vote_id,,,,,,,,,,,,,,,,,,,,,
6509,52701345,NaN,90.0,601560,4185,1053,4,66,0,20,...,51,"{""max"": {""maxOptPerc"": 73.333333333333329, ""ma...",None,"[3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 16, 17, 19, ...","[18, 45, 49, 58]","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53...",[],"{1: 34, 2: 2, 3: 10, 6: 5, 7: 6, 8: 5, 109: 3,...","{8: 1, 3: 1, 109: 1, 111: 1}",[]
6510,52709445,NaN,90.0,601560,4185,1053,2,88,0,0,...,51,"{""max"": {""maxOptPerc"": 97.777777777777771, ""ma...",None,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[18, 58]","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53...",[],"{1: 34, 2: 2, 3: 11, 5: 19, 6: 5, 7: 6, 8: 5, ...","{8: 1, 109: 1}",[]
6511,52717545,NaN,90.0,601560,4185,1053,5,64,21,0,...,51,"{""max"": {""maxOptPerc"": 71.111111111111114, ""ma...","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 49, 51...","[3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 16, 17, 19, ...","[18, 42, 45, 58, 80]","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53...","{108: 1, 5: 19, 111: 1}","{1: 34, 2: 2, 3: 10, 6: 5, 7: 6, 8: 3, 109: 3,...","{8: 3, 3: 1, 109: 1}",[]
6512,52725645,NaN,0.0,601560,4185,1053,5,26,59,0,...,51,"{""max"": {""maxOptPerc"": 65.555555555555557, ""ma...","[3, 5, 6, 7, 8, 9, 11, 13, 14, 16, 17, 19, 20,...","[2, 4, 10, 12, 15, 22, 23, 24, 25, 26, 34, 35,...","[18, 31, 58, 80, 82]","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53...","{112: 1, 1: 34, 3: 8, 6: 5, 7: 6, 8: 2, 109: 3}","{2: 2, 3: 3, 108: 1, 5: 19, 111: 1}","{8: 4, 109: 1}",[]
6513,52733745,NaN,90.0,601560,4185,1053,6,84,0,0,...,51,"{""max"": {""maxOptPerc"": 93.333333333333329, ""ma...",None,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[18, 45, 58, 80, 83, 91]","[2, 10, 12, 15, 23, 25, 26, 35, 36, 47, 51, 53...",[],"{1: 34, 2: 2, 3: 10, 5: 18, 6: 5, 7: 5, 8: 4, ...","{8: 2, 7: 1, 3: 1, 5: 1, 109: 1}",[]
